Setup, Kredensial, dan Download Model Clasification

In [3]:
# @title hitemanTTS.ipynb - Setup Awal dan Konfigurasi Gemini

# --- Instalasi Library ---
!pip install google-generativeai --upgrade -q
!pip install wave -q

import os
import shutil
import asyncio
import wave

import google.generativeai as genai
from google.generativeai import types as genai_types

print("--- hitemanTTS - Sel 1: Setup dan Konfigurasi ---")

# --- Mengambil API Key dari Colab Secrets ---
GEMINI_API_KEY = None # Default
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GEMINI_API_KEY:
        print("PERINGATAN (hitemanTTS): Secret 'GOOGLE_API_KEY' tidak ditemukan atau kosong di Colab Secrets.")
    else:
        print("Info (hitemanTTS): API Key Gemini berhasil diambil dari Colab Secrets.")
except ImportError:
    print("PERINGATAN (hitemanTTS): Modul userdata tidak ditemukan. Tidak dapat mengambil API Key dari Secrets.")
    print("  Ini normal jika dijalankan di luar Google Colab.")
except Exception as e_secret:
    print(f"Error (hitemanTTS): Terjadi kesalahan saat mengambil API Key dari Secrets: {e_secret}")


# Konfigurasi global google.generativeai dengan API Key
if GEMINI_API_KEY:
    try:
        genai.configure(api_key=GEMINI_API_KEY)
        print("Info (hitemanTTS): Konfigurasi global google.generativeai dengan API Key berhasil.")
    except Exception as e:
        print(f"ERROR (hitemanTTS): Gagal saat konfigurasi global google.generativeai - {e}")
        GEMINI_API_KEY = None # Anggap konfigurasi gagal jika ada error
else:
    print("PERINGATAN (hitemanTTS): GEMINI_API_KEY tidak diset atau tidak berhasil diambil. Fungsi Gemini tidak akan berfungsi.")

# Direktori untuk output audio TTS (digunakan oleh fungsi di notebook ini)
AUDIO_OUTPUT_DIR_TTS = "temp_hiteman_audio_from_tts_notebook"
if os.path.exists(AUDIO_OUTPUT_DIR_TTS):
    shutil.rmtree(AUDIO_OUTPUT_DIR_TTS)
os.makedirs(AUDIO_OUTPUT_DIR_TTS, exist_ok=True)
print(f"Info (hitemanTTS): Direktori output audio diatur ke: {AUDIO_OUTPUT_DIR_TTS}")

print("\n--- Selesai: hitemanTTS ---")

--- hitemanTTS - Sel 1: Setup dan Konfigurasi ---
Info (hitemanTTS): API Key Gemini berhasil diambil dari Colab Secrets.
Info (hitemanTTS): Konfigurasi global google.generativeai dengan API Key berhasil.
Info (hitemanTTS): Direktori output audio diatur ke: temp_hiteman_audio_from_tts_notebook

--- Selesai: hitemanTTS ---


In [4]:
# @title hitemanTTS.ipynb - Definisi Layanan Model Gemini (Teks & TTS)

print("--- hitemanTTS - Layanan Model Gemini ---")

# Variabel global yang akan diisi/digunakan oleh fungsi-fungsi di bawah
gemini_text_gen_model_instance = None # Untuk model generasi teks
gemini_text_model_is_ready = False    # Status inisialisasi model teks

# Nama model TTS Native Gemini
# https://ai.google.dev/gemini-api/docs/models/tts-models
# Contoh: "gemini-1.5-flash-tts-001" atau "gemini-1.5-pro-tts-001"
DEFAULT_NATIVE_GEMINI_TTS_MODEL_NAME = "models/gemini-2.5-flash-preview-tts"
print(f"Info (hitemanTTS): Akan menggunakan model TTS Native: {DEFAULT_NATIVE_GEMINI_TTS_MODEL_NAME}")


# --- Fungsi Inisialisasi Model Teks Gemini ---
def initialize_gemini_language_model():
    """
    Menginisialisasi model Gemini yang akan digunakan untuk generasi skrip teks.
    Fungsi ini harus dipanggil sebelum menggunakan generate_therapy_script.
    """
    global gemini_text_gen_model_instance, gemini_text_model_is_ready

    if gemini_text_model_is_ready:
        # print("Info (hitemanTTS): Model teks Gemini sudah diinisialisasi sebelumnya.")
        return True

    if not GEMINI_API_KEY: # Variabel global
        print("ERROR (hitemanTTS): GEMINI_API_KEY tidak tersedia. Tidak dapat inisialisasi Model Teks Gemini.")
        return False

    try:
        # Pilih model yang sesuai untuk generasi teks (bukan model TTS)
        text_model_name = "models/gemini-1.5-flash"

        gemini_text_gen_model_instance = genai.GenerativeModel(
            model_name=text_model_name,
            generation_config={ # Konfigurasi generasi teks
                "temperature": 0.7,
                "top_p": 0.9,
                "max_output_tokens": 400
            },
            safety_settings=[ # Sesuaikan safety settings sesuai kebutuhan
                {"category": genai_types.HarmCategory.HARM_CATEGORY_HARASSMENT, "threshold": genai_types.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE},
                {"category": genai_types.HarmCategory.HARM_CATEGORY_HATE_SPEECH, "threshold": genai_types.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE},
                {"category": genai_types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT, "threshold": genai_types.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE},
                {"category": genai_types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT, "threshold": genai_types.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE},
            ]
        )
        gemini_text_model_is_ready = True
        print(f"Info (hitemanTTS): Model teks Gemini ({text_model_name}) berhasil diinisialisasi.")
        return True
    except Exception as e:
        print(f"ERROR (hitemanTTS): Gagal saat inisialisasi model teks Gemini - {e}")
        gemini_text_model_is_ready = False
        return False

# --- Fungsi untuk Membuat Prompt Skrip Terapi ---
def create_custom_therapy_prompt(emotion_label: str) -> str:
    """Membuat prompt yang disesuaikan untuk Gemini agar menghasilkan skrip terapi."""
    emotion_description_map = {
        "happy": "merasa sangat senang dan bahagia",
        "sadness": "merasa sangat sedih, mungkin kehilangan semangat atau merasa hampa",
        "fear": "merasa ketakutan, cemas, atau khawatir berlebihan tentang sesuatu", # Menggabungkan fear & khawatir
        "stress": "merasa tertekan, tegang, atau kewalahan oleh tuntutan"
    }
    # Deskripsi default jika emosi tidak ada di map
    user_current_feeling = emotion_description_map.get(emotion_label.lower(), f"sedang merasakan emosi '{emotion_label}' yang cukup kuat")

    prompt = f"""
Anda adalah "HiTeman", AI pendamping virtual yang sangat empatik, lembut, dan suportif.
Pengguna sedang {user_current_feeling}.

Tugas Anda adalah membuat skrip audio terapi singkat (4-7 kalimat alami) untuk pengguna ini.
Panduan ketat:
1.  **Validasi Emosi (1-2 kalimat)**: Akui perasaan pengguna dengan hangat. Contoh: "Saya dengar kamu sedang merasa {emotion_label}, ya..."
2.  **Pendekatan ACT Ringan (1-2 kalimat)**: Pilih salah satu: Penerimaan (Izinkan perasaan hadir), Mindfulness (Fokus napas/sensasi tubuh), atau Metafora sederhana (Perasaan seperti awan/ombak).
3.  **Bahasa (Seluruh Skrip)**: Bahasa Indonesia yang sangat menenangkan, lembut, empatik. Hindari menggurui/kaku/formal. Jadilah teman peduli.
4.  **Penguatan dan Harapan (1-2 kalimat)**: Akhiri dengan kalimat positif dan memberdayakan. Contoh: "Ingat, kamu tidak sendirian." atau "Kamu lebih kuat dari yang kamu bayangkan."

Contoh Inspirasi (Kembangkan kreativitas empatik Anda):
*   'sadness': "Aku memahami kamu sedang merasa sangat sedih. Tak apa membiarkan kesedihan itu terasa. Seperti langit kelabu, perasaan ini pun akan berlalu. Ketahuilah kamu berharga, HiTeman ada di sini."
*   'fear'/'khawatir': "Rasanya pasti sangat tidak nyaman ya ketika ketakutan atau kekhawatiran itu datang. Mari coba ambil satu napas panjang... rasakan... hembuskan perlahan. Ingat, kamu aman saat ini, dan kamu punya kemampuan melewati ini."
*   'stress': "Aku mengerti kamu sedang merasa sangat tertekan. Beban itu pasti terasa berat. Coba kita fokus sejenak pada satu hal kecil yang bisa kamu kendalikan saat ini. Kamu melakukan yang terbaik, dan itu sudah cukup."
*   'happy': "Senang sekali mendengar kamu sedang merasa bahagia! Nikmati perasaan indah ini sepenuhnya, kamu pantas mendapatkannya. Semoga kebahagiaan ini terus mewarnai harimu ya."

PENTING: HINDARI diagnosis, janji kesembuhan, jargon kompleks. Fokus pada kehangatan, validasi, dukungan emosional awal.
Sekarang, buatkan skrip terapi untuk pengguna yang sedang {user_current_feeling}.
"""
    return prompt

# --- Fungsi Generasi Skrip Teks dari Model Bahasa Gemini ---
async def generate_therapy_script(emotion_label: str) -> str:
    if not gemini_text_model_is_ready:
        if not initialize_gemini_language_model():
            raise RuntimeError("Gagal menginisialisasi model teks Gemini yang diperlukan untuk skrip.")
    if not gemini_text_gen_model_instance:
         raise RuntimeError("Model teks Gemini tidak siap atau belum diinisialisasi dengan benar.")
    try:
        custom_prompt = create_custom_therapy_prompt(emotion_label)
        response = await asyncio.to_thread(gemini_text_gen_model_instance.generate_content, custom_prompt)
        script_text_content = ""
        if hasattr(response, 'text') and response.text:
            script_text_content = response.text
        elif response.candidates and response.candidates[0].content and response.candidates[0].content.parts:
            script_text_content = "".join(part.text for part in response.candidates[0].content.parts if hasattr(part, 'text'))
        else:
            print(f"Warning (hitemanTTS): Struktur respons teks Gemini tidak biasa: {response}")
            script_text_content = f"Hi Teman, saya mengerti kamu sedang merasa {emotion_label}. Ingatlah bahwa semua perasaan itu wajar dan kamu tidak sendirian dalam menghadapinya. HiTeman ada di sini untukmu."
        cleaned_script = script_text_content.replace("```text", "").replace("```", "").strip()
        return cleaned_script
    except Exception as e:
        print(f"ERROR (hitemanTTS): Terjadi kesalahan saat menghasilkan skrip teks dari Gemini - {e}")
        raise RuntimeError(f"Gagal menghasilkan skrip terapi teks: {str(e)}")

# --- Fungsi utilitas untuk menyimpan file WAV ---
def _save_pcm_to_wave_file(output_filename: str, pcm_audio_data: bytes,
                           num_channels: int = 1, audio_sample_rate: int = 24000,
                           sample_width_in_bytes: int = 2):
    """Internal helper untuk menyimpan data audio PCM mentah ke file format WAV."""
    # Pastikan direktori output tempat file akan disimpan sudah ada
    target_directory = os.path.dirname(output_filename)
    if not os.path.exists(target_directory):
        os.makedirs(target_directory, exist_ok=True) # Buat jika belum ada

    try:
        with wave.open(output_filename, "wb") as wf:
            wf.setnchannels(num_channels)           # Mono adalah default untuk TTS ini
            wf.setsampwidth(sample_width_in_bytes)  # 2 bytes = 16-bit PCM audio
            wf.setframerate(audio_sample_rate)      # 24kHz adalah sample rate default Gemini TTS
            wf.writeframes(pcm_audio_data)
        print(f"Info (hitemanTTS): Audio berhasil disimpan sebagai file WAV di: {output_filename}")
    except Exception as e_wave:
        print(f"ERROR (hitemanTTS): Gagal menyimpan file WAV '{output_filename}' - {e_wave}")
        raise RuntimeError(f"Gagal menyimpan file audio WAV: {str(e_wave)}")


# --- Fungsi Generasi Audio menggunakan Native Gemini TTS ---
async def generate_audio_with_native_tts(
    text_for_speech: str
) -> str:
    if not GEMINI_API_KEY: # Variabel global dari Sel 1 notebook ini
        raise RuntimeError("GEMINI_API_KEY tidak tersedia. TTS Native Gemini tidak dapat berfungsi.")

    try:
        # Prompt instruksional untuk TTS
        tts_instructional_prompt = f"Tolong bacakan teks berikut dalam Bahasa Indonesia dengan suara yang tenang, lembut, penuh perhatian, dan sangat empatik, seolah-olah untuk sesi terapi audio yang menenangkan: \"{text_for_speech}\""

        # --- PERCOBAAN KONFIGURASI TTS: MENGIRIM DICTIONARY LANGSUNG ---

        generation_config_dict_for_tts = {
            "response_modalities": ["AUDIO"] # String literal "AUDIO"
        }

        native_tts_model = genai.GenerativeModel(DEFAULT_NATIVE_GEMINI_TTS_MODEL_NAME)

        print(f"Info (hitemanTTS): Mengirim TTS ke model '{DEFAULT_NATIVE_GEMINI_TTS_MODEL_NAME}' dengan config dict: {generation_config_dict_for_tts}")

        response_object = await asyncio.to_thread(
            native_tts_model.generate_content,
            contents=tts_instructional_prompt,
            generation_config=generation_config_dict_for_tts # Kirim dictionary langsung
        )

        # Pastikan ekstraksi audio dan penyimpanan file WAV sudah benar
        if not (response_object.candidates and
                response_object.candidates[0].content and
                response_object.candidates[0].content.parts and
                hasattr(response_object.candidates[0].content.parts[0], 'inline_data') and
                hasattr(response_object.candidates[0].content.parts[0].inline_data, 'data')):
            # Log detail respons jika struktur tidak sesuai
            error_detail_from_response = f"Respons lengkap: {response_object}"
            print(f"ERROR (hitemanTTS): Respons TTS dari Gemini tidak valid atau tidak mengandung data audio. {error_detail_from_response}")
            if hasattr(response_object, 'prompt_feedback') and response_object.prompt_feedback:
                print(f"  Prompt Feedback: {response_object.prompt_feedback}")
            raise ValueError(f"Respons TTS dari Gemini tidak mengandung data audio PCM yang diharapkan. {error_detail_from_response}")

        pcm_audio_data_bytes = response_object.candidates[0].content.parts[0].inline_data.data
        unique_hash_for_filename = abs(hash(text_for_speech)) % (10**9)
        output_wav_filepath = os.path.join(AUDIO_OUTPUT_DIR_TTS, f"hiteman_therapy_audio_native_{unique_hash_for_filename}.wav") # AUDIO_OUTPUT_DIR_TTS dari Sel 1

        _save_pcm_to_wave_file(output_wav_filepath, pcm_audio_data_bytes,
                               num_channels=1, audio_sample_rate=24000, sample_width_in_bytes=2)

        return output_wav_filepath

    except Exception as e:
        print(f"ERROR (hitemanTTS): Terjadi kesalahan fatal saat generasi audio Native Gemini TTS - {type(e).__name__}: {e}")
        # Tambahkan logging lebih detail jika ada informasi spesifik dari exception
        if hasattr(e, 'response') and e.response: # Untuk error dari library requests
            print(f"  Detail HTTP response (jika ada): {e.response.text if hasattr(e.response, 'text') else e.response}")
        # Untuk error API Google, biasanya ada di message atau args
        if hasattr(e, 'message'): print(f"  Pesan Error API (atribut .message): {e.message}")
        if e.args: print(f"  Args Error API: {e.args}")

        raise RuntimeError(f"Gagal menghasilkan audio dari Native Gemini TTS: {type(e).__name__} - {str(e)}")


# Inisialisasi model teks saat notebook/modul ini dimuat atau dijalankan.
if not gemini_text_model_is_ready:
    initialize_gemini_language_model()

print("--- Selesai: hitemanTTS - Sel 2 ---")

--- hitemanTTS - Layanan Model Gemini ---
Info (hitemanTTS): Akan menggunakan model TTS Native: models/gemini-2.5-flash-preview-tts
Info (hitemanTTS): Model teks Gemini (models/gemini-1.5-flash) berhasil diinisialisasi.
--- Selesai: hitemanTTS - Sel 2 ---
